# Задание 1. Bootstrap

В этом задании используйте датасет breast_cancer — классический датасет для задачи бинарной классификации. Обучите модели:

 - `DecisionTreeClassifier`
 - `RandomForestClassifier`
 - `LigthGBMClassifier`
 - `SVC`
 - `BaggingClassifier` с базовым класификатором `SVC`.

Параметры моделей можете оставить по умолчанию или задать сами.

Для каждой модели посчитайте [корреляцию Мэтьюса](https://en.wikipedia.org/wiki/Phi_coefficient) — метрику для оценки качества бинарной классификации, в частности, устойчивую к дисбалансу классов, ([`sklearn.metrics.matthews_corrcoef`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.matthews_corrcoef.html), подробнее почитать про его пользу можно [здесь](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7)) — для предсказанного ею класса и реального.

С помощью bootstrap-подхода постройте 90% доверительные интервалы для качества полученных моделей. Используйте функцию `bootstrap_metric()` из лекции.

Постройте [боксплоты](https://seaborn.pydata.org/generated/seaborn.boxplot.html) для качества полученных моделей.

Импорт необходимых библиотек:

In [ ]:
import lightgbm
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.datasets
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

Загрузка датасета

In [ ]:
breast_cancer = sklearn.datasets.load_breast_cancer()
print(breast_cancer.DESCR)

In [ ]:
x = breast_cancer.data
y = breast_cancer.target
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)

In [ ]:
# Your code here

models = {}
models["rf"] = RandomForestClassifier(n_estimators=200)
models["svc"] = SVC()
models["dt"] = DecisionTreeClassifier()
models["gb"] = lightgbm.LGBMClassifier(verbose=-1)
models["bagged svc"] = BaggingClassifier(estimator=SVC())

In [ ]:
for name, model in models.items():
    print(f"Fitting {name}")
    model.fit(x_train, y_train)

In [ ]:
predictions = {}
for name, model in models.items():
    predictions[name] = model.predict(x_test)

In [ ]:
from sklearn.metrics import matthews_corrcoef

for name, pred in predictions.items():
    qual = matthews_corrcoef(y_pred=pred, y_true=y_test)
    print(f"{name} MCC: {qual:.02f}")

In [ ]:
def bootstrap_metric(x, y, metric_fn, samples_cnt=1000, alpha=0.05, random_state=42):
    size = len(x)

    np.random.seed(random_state)
    b_metric = np.zeros(samples_cnt)
    for it in range(samples_cnt):
        poses = np.random.choice(x.shape[0], size=x.shape[0], replace=True)

        x_boot = x[poses]
        y_boot = y[poses]

        m_val = metric_fn(x_boot, y_boot)
        b_metric[it] = m_val

    return b_metric

In [ ]:
alpha = 0.10
boot_mcc = {}
for name, pred in predictions.items():
    boot = bootstrap_metric(pred, y_test, metric_fn=matthews_corrcoef, samples_cnt=200)
    # metric_fn=lambda x, y: matthews_corrcoef(y_pred=x, y_true=y)) - unnecessary code
    boot_mcc[name] = boot
    print(f"{name} MCC: ", np.quantile(boot, q=[alpha / 2, 1 - alpha / 2]))

In [ ]:
mcc_table = pd.DataFrame(boot_mcc)
mcc_table = mcc_table.melt(
    value_vars=mcc_table.columns, value_name="MCC", var_name="model"
)
plt.figure(figsize=(10, 6))
sns.boxplot(data=mcc_table, x="model", y="MCC")
plt.title("Models MCC", size=30)
plt.ylabel("MCC", size=25)
plt.xticks(size=20)
plt.xlabel("")
plt.show()

Сделайте вывод о том, какие модели работают лучше.

**Напишите вывод**

## Формат результата

График с демонстрацией корреляции Мэтьюса для следующих моделей:

 - `DecisionTreeClassifier`
 - `RandomForestClassifier`
 - `LigthGBMClassifier`
 - `SVC`
 - `BaggingClassifier` с базовым класификатором `SVC`

Пример графика:

<img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.0/Exercises/EX03/result_1_task_ex03.png" width="600">

# Задание 2. Дисбаланс классов

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -qU imbalanced-learn

In [ ]:
import imblearn
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import (
    train_test_split,
    KFold,
    StratifiedKFold,
    cross_validate,
)

Важно обращать внимание на сбалансированность классов в наборе.
Предположим, у нас есть некоторый набор данных со следующими метками классов:

In [ ]:
real_labels = [1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]

В наборе 16 объектов относятся к классу 0, а 5 — к классу 1.

Мы обучили две модели. Первая всегда выдает 0:

In [ ]:
model1_res = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

Вторая сумела обнаружить некоторую закономерность в признаках:

In [ ]:
model2_res = [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1]

Рассчитаем точность Accuracy (см. лекцию 1) для этих моделей:

In [ ]:
print("Accuracy for model1: ", accuracy_score(real_labels, model1_res))
print("Accuracy for model2: ", accuracy_score(real_labels, model2_res))

Accuracy нельзя использовать, если данные не сбалансированы. Для несбалансированных данных необходимо использовать свои метрики и модели. Одной из таких метрик является balanced accuracy. При вычислении данной метрики считается полнота (recall) отдельно для каждого класса и вычисляется среднее значение:

In [ ]:
# Balanced accuracy for model1 = (16/16+0/5)/2 = 0.5
print(
    "Balanced accuracy for model1: ", balanced_accuracy_score(real_labels, model1_res)
)
# Balanced accuracy for model2 = (12/16+4/5)/2 = 0.775
print(
    "Balanced accuracy for model2: ", balanced_accuracy_score(real_labels, model2_res)
)

**Всегда проверяйте**, являются ли ваши данные сбалансированными и могут ли выбранные для оценки модели метрики работать с несбалансированными классами.

Загрузим датасет с различными биомаркерами пациентов с меланомой (обезличенный, информации о пациентах нет) и переменной, содержащей 1, если пациент ответил на иммунотерапию (терапия помогла пациенту и произошло уменьшение размеров опухоли), и 0, если не ответил. Количество пациентов, отвечающих на терапию, сильно меньше пациентов, которым терапия не помогает, поэтому предсказание ответа пациента на терапию на основании биомаркеров — актуальная задача в онкологии. В данном задании вам предстоит попробовать её решить.

In [ ]:
cancer = pd.read_table(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/Cancer_dataset_2.tsv",
    index_col="sample_id",
)
display(cancer.head())

# split the data on features (x) and dependant variable (y)
y = cancer["Response"]
x = cancer.drop("Response", axis=1)
print("\nNumber of patients responded to immunotherapy:")
display(y.value_counts())

В данном случае имеет место несбалансированность классов в наборе данных: пациентов, ответивших на терапию, гораздо меньше.

Есть два способа работы с несбалансированными по классам данными. Первый способ &mdash; это получение стратифицированных выборок. Необходимо иметь одинаковую долю образцов каждого класса в тренировочной и тестовой выборках, иначе возникает риск получения смещённых выборок, что приводит к некорректной оценке качества модели. Второй способ &mdash; это использование специальных алгоритмов, учитывающих несбалансированность классов.


В данном задании вам нужно продемонстрировать эффективность различных подходов  работы с несбалансированными выборками. Для этого вы будете использовать три модели, представленные ниже:

1. [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), библиотека sklearn
2. [`RandomForestClassifier` с балансировкой классов](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), библиотека sklearn — меняет стандартный вес каждого класса, равный 1, на долю класса во входных данных (см. `class_weight`).
3. [`BalancedRandomForestClassifier`](https://imbalanced-learn.org/stable/references/generated/imblearn.ensemble.BalancedRandomForestClassifier.html), библиотека imblearn — сэмплирует псевдовыборки таким образом, что в каждой псевдовыборке, которая подается на вход модели, баланс классов оказывается "выправлен".

Оцените эффективность подходов с помощью кросс-валидации, производя разбиение с учетом репрезентации классов и без него. В качестве метрики, отображающей эффективность модели, используйте значения `accuracy` и `balanced_accuracy`. Проинтерпретируйте результаты.

In [ ]:
?imblearn.ensemble.BalancedRandomForestClassifier

In [ ]:
?cross_validate

Объекты, принадлежащие разным классам, распределены неравномерно. Для адекватной работы `cross_validate` нужно перемешать данные. Для этого используйте флаг `shuffle=True`.

In [ ]:
# Your code here
rng = np.random.RandomState(42)

cvs = {
    "cv_rand": KFold(n_splits=3, shuffle=True, random_state=rng),
    "sv_strat": StratifiedKFold(n_splits=3, shuffle=True, random_state=rng),
}

models = {
    "rf_sklearn": RandomForestClassifier(
        n_estimators=500, max_depth=10, random_state=rng
    ),
    "rf_sklearn_balanced": RandomForestClassifier(
        n_estimators=500, max_depth=10, class_weight="balanced", random_state=rng
    ),
    "rf_imblearn": imblearn.ensemble.BalancedRandomForestClassifier(
        n_estimators=500, max_depth=10, random_state=rng
    ),
}

for name_model, model in models.items():
    for name_cv, cv in cvs.items():
        scores = cross_validate(
            model, X=x, y=y, scoring=("accuracy", "balanced_accuracy"), cv=cv, n_jobs=-1
        )
        print(
            f'{name_model} {name_cv}: accuracy = {scores["test_accuracy"].mean()}, balanced_accuracy = {scores["test_balanced_accuracy"].mean()}'
        )

In [ ]:
cv = KFold(n_splits=3, shuffle=True, random_state=rng)

model = RandomForestClassifier(n_estimators=500, max_depth=10)
scores = cross_validate(
    model, X=x, y=y, scoring=("accuracy", "balanced_accuracy"), cv=cv, n_jobs=-1
)

for name_model, model in models.items():
    for name_cv, cv in cvs.items():
        scores = cross_validate(
            model, X=x, y=y, scoring=("accuracy", "balanced_accuracy"), cv=cv, n_jobs=-1
        )

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=42, test_size=0.3
)

In [ ]:
def bootstrap_metric(x, y, metric_fn, samples_cnt=1000, random_state=42):
    np.random.seed(random_state)
    b_metric = np.zeros(samples_cnt)
    for it in range(samples_cnt):
        poses = np.random.choice(x.shape[0], size=x.shape[0], replace=True)

        x_boot = x[poses]
        y_boot = y[poses]
        m_val = metric_fn(x_boot, y_boot)
        b_metric[it] = m_val

    return b_metric

In [ ]:
alpha = 0.1
boot = {}
models = {
    "rf_sklearn": RandomForestClassifier(
        n_estimators=500, max_depth=10, random_state=42
    ),
    "rf_sklearn_balanced": RandomForestClassifier(
        n_estimators=500, max_depth=10, class_weight="balanced", random_state=42
    ),
    "rf_imblearn": imblearn.ensemble.BalancedRandomForestClassifier(
        n_estimators=500,
        max_depth=10,
        random_state=42,
        sampling_strategy="all",
        replacement=True,
    ),
}

for name_model, model in models.items():
    model.fit(x_train, y_train)
    pred = model.predict(x_test)
    boot[name_model] = bootstrap_metric(
        pred, y_test, metric_fn=lambda x, y: balanced_accuracy_score(y_true=y, y_pred=x)
    )

In [ ]:
plt.figure(figsize=(16, 6))
sns.boxplot(
    y=np.concatenate(
        [boot["rf_sklearn"], boot["rf_sklearn_balanced"], boot["rf_imblearn"]]
    ),
    x=["rf_sklearn"] * 1000 + ["rf_sklearn_balanced"] * 1000 + ["rf_imblearn"] * 1000,
)
plt.ylabel("Balanced accuracy", size=20)
plt.tick_params(axis="both", which="major", labelsize=14)
plt.show()

Какая модель лучше справляется с дисбалансом классов?

**Напишите вывод**

Выводы:
1. Оценка качества модели на стратифицированной выборке более адекватна, что можно заметить по росту balanced_accuracy для моделей, обученных и протестированных на стратифицированной выборке
2. Модели, учитывающие дисбаланс классов, лучше справляются с задачей классификации при наличии дисбаланса.

## Формат результата

Получить значения `accuracy` и `balanced_accuracy`для моделей:
1. `RandomForestClassifier`, библиотека sklearn;
2. `RandomForestClassifier с балансировкой классов`, библиотека sklearn;
3. `BalancedRandomForestClassifier`, библиотека imblearn.

# Задание 3. Разные типы бустингов

В этом задании будем использовать датасет с рейтингом блюд по некоторым характеристикам.

В некоторых реализациях градиентного бустинга есть возможность использовать другой метод обучения. Например, в XGB есть тип `dart`, а в lgbm — `goss`. Это позволяет составлять более эффективные ансамбли.

Используя кросс-валидацию (используйте 3 фолда), обучите модели:
* CatboostRegressor
* XGBRegressor
* LGBMRegressor

Сохраните модель на каждом фолде и посчитайте `mse` для тестовой выборки, используя модель с каждого фолда. Получите предсказания всех 9 моделей на тестовой выборке и усредните их. Затем посчитайте `mse` для усредненных предсказаний.

Напишите выводы о полученном качестве моделей.

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -q catboost

In [ ]:
import xgboost
import catboost
import lightgbm
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, KFold

Загрузка датасета:

In [ ]:
recipies = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/recipes.csv"
)
recipies

In [ ]:
y = recipies["rating"]
x = recipies.drop(["rating"], axis=1)

x_train_all, x_test, y_train_all, y_test = train_test_split(
    x.values, y.values, train_size=0.7, random_state=42
)

In [ ]:
models = {}

kf = KFold(n_splits=3, shuffle=True, random_state=42)

for num_fold, (train_index, val_index) in enumerate(kf.split(x)):
    x_train, x_val = x.iloc[train_index], x.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = catboost.CatBoostRegressor(
        iterations=500,
        learning_rate=0.1,
        random_state=42,
        verbose=0,
    )

    model.fit(x_train, y_train)

    models[f"Catboost_{num_fold+1}"] = model

cb_test_pred = np.mean(
    [
        models["Catboost_1"].predict(x_test),
        models["Catboost_2"].predict(x_test),
        models["Catboost_3"].predict(x_test),
    ],
    axis=0,
)


print("Catboost 1 fold mse score: ", mse(y_test, models["Catboost_1"].predict(x_test)))
print("Catboost 2 fold mse score: ", mse(y_test, models["Catboost_2"].predict(x_test)))
print("Catboost 3 fold mse score: ", mse(y_test, models["Catboost_3"].predict(x_test)))
print("Catboost all folds mean mse score: ", mse(y_test, cb_test_pred))

In [ ]:
for num_fold, (train_index, val_index) in enumerate(kf.split(x)):
    x_train, x_val = x.iloc[train_index], x.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = xgboost.XGBRegressor(
        n_estimators=500,
        learning_rate=0.1,
        max_depth=5,
        random_state=42,
        min_child_weight=9,
        n_jobs=-1,
        objective="reg:squarederror",
        booster="dart",
        rate_drop=0.1,
        one_drop=1,
        verbosity=0,
    )

    model.fit(x_train, y_train)

    models[f"xgb_{num_fold+1}"] = model

xgb_test_pred = np.mean(
    [
        models["xgb_1"].predict(x_test),
        models["xgb_2"].predict(x_test),
        models["xgb_3"].predict(x_test),
    ],
    axis=0,
)


print("xgb 1 fold mse score: ", mse(y_test, models["xgb_1"].predict(x_test)))
print("xgb 2 fold mse score: ", mse(y_test, models["xgb_2"].predict(x_test)))
print("xgb 3 fold mse score: ", mse(y_test, models["xgb_3"].predict(x_test)))
print("xgb all folds mean mse score: ", mse(y_test, xgb_test_pred))

In [ ]:
for num_fold, (train_index, val_index) in enumerate(kf.split(x)):
    x_train, x_val = x.iloc[train_index], x.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = lightgbm.LGBMRegressor(
        n_estimators=500,
        learning_rate=0.1,
        max_depth=-1,
        num_leaves=2**5,
        random_state=42,
        min_child_weight=9,
        n_jobs=-1,
        boosting_type="goss",
        verbose=-1,
    )

    model.fit(x_train, y_train)

    models[f"lgbm_{num_fold+1}"] = model

lgbm_test_pred = np.mean(
    [
        models["lgbm_1"].predict(x_test),
        models["lgbm_2"].predict(x_test),
        models["lgbm_3"].predict(x_test),
    ],
    axis=0,
)


print("lgbm 1 fold mse score: ", mse(y_test, models["lgbm_1"].predict(x_test)))
print("lgbm 2 fold mse score: ", mse(y_test, models["lgbm_2"].predict(x_test)))
print("lgbm 3 fold mse score: ", mse(y_test, models["lgbm_3"].predict(x_test)))
print("lgbm all folds mean mse score: ", mse(y_test, lgbm_test_pred))

In [ ]:
all_preds = []
for model in models:
    print(model)
    all_preds.append(models[model].predict(x_test))

print("ensemble mse: ", mse(y_test, np.mean(all_preds, axis=0)))

## Формат результата

Получить значения MSE для всех моделей и значение MSE, усреднив предсказания всех моделей. Написать вывод.


# Задание 4. Подбор гиперпараметров

В этом задании нужно подобрать параметры для бустинга `CatBoostRegressor`, используя библиотеку `optuna`. И улучшить результат по сравнению со стандартными параметрами.

Список параметров для подбора:

* `depth`
* `iterations`
* `learning_rate`
* `colsample_bylevel`
* `subsample`
* `l2_leaf_reg`
* `min_data_in_leaf`
* `max_bin`
* `random_strength`
* `bootstrap_type`

**Важно!** *Подбирать параметры нужно на валидационной выборке*

Установка и импорт необходимых библиотек:

In [ ]:
!pip install -q catboost
!pip install -q optuna

In [ ]:
import optuna
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from optuna.samplers import RandomSampler
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, KFold

Загрузка датасета:

In [ ]:
recipies = pd.read_csv(
    "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/recipes.csv"
)
recipies

In [ ]:
y = recipies["rating"]
x = recipies.drop(["rating"], axis=1)

x_train, x_test, y_train, y_test = train_test_split(
    x.values, y.values, train_size=0.7, random_state=42
)

In [ ]:
model = CatBoostRegressor(random_seed=42)

model.fit(
    x_train,
    y_train,
    eval_set=(x_test, y_test),
    verbose=200,
    use_best_model=True,
    plot=False,
    early_stopping_rounds=100,
)

print("\nmse_score before tuning: ", mse(y_test, model.predict(x_test)))

In [ ]:
def objective(trial):
    models = {}
    params = {
        "depth": trial.suggest_int("depth", 4, 12),
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 5),
        "max_bin": trial.suggest_int("max_bin", 32, 256),
        "random_strength": trial.suggest_float("random_strength", 0, 2),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bernoulli", "MVS"]
        ),
        "task_type": "CPU",
        "thread_count": -1,
        "random_seed": 42,
        "early_stopping_rounds": 50,
    }

    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    for num_fold, (train_index, val_index) in enumerate(kf.split(x_train_all)):
        x_train, x_val = x_train_all[train_index], x_train_all[val_index]
        y_train, y_val = y_train_all[train_index], y_train_all[val_index]

        model = CatBoostRegressor(**params)

        model.fit(
            x_train,
            y_train,
            eval_set=(x_val, y_val),
            verbose=0,
            use_best_model=True,
            plot=False,
        )

        models[f"Catboost_{num_fold+1}"] = model

    cb_test_pred = np.mean(
        [
            models["Catboost_1"].predict(x_val),
            models["Catboost_2"].predict(x_val),
            models["Catboost_3"].predict(x_val),
        ],
        axis=0,
    )
    result_score = mse(y_val, cb_test_pred)

    return result_score


# Create "exploration"
study = optuna.create_study(
    direction="minimize", study_name="Optimizer", sampler=RandomSampler(42)
)

study.optimize(
    objective, n_trials=100
)  # The more iterations, the higher the chances of catching the most optimal hyperparameters

In [ ]:
study.best_params

In [ ]:
model = CatBoostRegressor(**study.best_params, random_seed=42)

model.fit(
    x_train,
    y_train,
    eval_set=(x_test, y_test),
    verbose=200,
    use_best_model=True,
    plot=False,
    early_stopping_rounds=100,
)

print("\nmse_score after tuning: ", mse(y_test, model.predict(x_test)))

In [ ]:
optuna.visualization.plot_optimization_history(study)

## Формат результата

Значение `mse` с подобранными параметрами меньше, чем при стандартных параметрах.